In [1]:
import pandas as pd
import numpy as np
import urllib
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
#import seaborn as sns

import geopandas as gpd
import shapely as shp
from shapely.geometry import Point, Polygon

%matplotlib inline

url = "Arrest_Data_from_2010_to_2019.csv"
arrest_data = pd.read_csv(url)

In [2]:
pd.set_option('display.max_columns', None)
arrest_data.head()


,Report ID,Report Type,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Disposition Description,Address,Cross Street,LAT,LON,Location,Booking Date,Booking Time,Booking Location,Booking Location Code
0,2377805,BOOKING,06/22/2010,1845.0,16,Foothill,1664,46,F,W,6.0,Larceny,F,487(A)PC,GRAND THEFT MONEY/PROPERTY > $400,MISDEMEANOR COMPLAINT FILED,PENDLETON,GLENOAKS,34.2375,-118.3745,POINT (-118.3745 34.2375),06/22/2010 12:00:00 AM,2108.0,VALLEY - JAIL DIV,4279.0
1,121920046,RFC,09/28/2012,930.0,19,Mission,1998,60,M,H,NaN,NaN,M,25620(A)BP,NaN,MISDEMEANOR COMPLAINT FILED,7600 WOODMAN AV,NaN,34.2111,-118.4309,POINT (-118.4309 34.2111),NaN,NaN,NaN,NaN
2,101820989,RFC,10/20/2010,700.0,18,Southeast,1829,14,M,H,8.0,Other Assaults,M,242PC,BATTERY,NaN,7200 QUAIL DR,NaN,34.1025,-118.2091,POINT (-118.2091 34.1025),NaN,NaN,NaN,NaN
3,90712341,RFC,03/10/2011,940.0,7,Wilshire,776,45,M,B,18.0,Drunkeness,M,41.27CLAMC,DRINKING IN PUBLIC,MISDEMEANOR COMPLAINT FILED,4500 W WASHINGTON BL,NaN,34.0399,-118.3375,POINT (-118.3375 34.0399),NaN,NaN,NaN,NaN
4,121909585,RFC,04/01/2012,1715.0,19,Mission,1993,37,M,H,NaN,NaN,M,25620(A)BP,NaN,MISDEMEANOR COMPLAINT FILED,8100 SEPULVEDA PL,NaN,34.2208,-118.4662,POINT (-118.4662 34.2208),NaN,NaN,NaN,NaN


In [3]:
arrest_data.columns

Index(['Report ID', 'Report Type', 'Arrest Date', 'Time', 'Area ID',
       'Area Name', 'Reporting District', 'Age', 'Sex Code', 'Descent Code',
       'Charge Group Code', 'Charge Group Description', 'Arrest Type Code',
       'Charge', 'Charge Description', 'Disposition Description', 'Address',
       'Cross Street', 'LAT', 'LON', 'Location', 'Booking Date',
       'Booking Time', 'Booking Location', 'Booking Location Code'],
      dtype='object')

In [4]:
arrest_data["Charge Group Description"].unique()

array(['Larceny', nan, 'Other Assaults', 'Drunkeness',
       'Prostitution/Allied', 'Miscellaneous Other Violations',
       'Disturbing the Peace', 'Weapon (carry/poss)', 'Liquor Laws',
       'Gambling', 'Aggravated Assault', 'Narcotic Drug Laws',
       'Disorderly Conduct', 'Pre-Delinquency', 'Sex (except rape/prst)',
       'Burglary', 'Fraud/Embezzlement', 'Driving Under Influence',
       'Robbery', 'Moving Traffic Violations', 'Receive Stolen Property',
       'Vehicle Theft', 'Rape', 'Against Family/Child',
       'Forgery/Counterfeit', 'Non-Criminal Detention', 'Homicide',
       'Federal Offenses'], dtype=object)

In [5]:
#Type of Crime Distribution
arrest_data["Charge Group Description"].value_counts()

#DUI is a one of the most common crimes

Miscellaneous Other Violations    247309
Narcotic Drug Laws                164935
Driving Under Influence           121528
Drunkeness                        116190
Aggravated Assault                 86713
Larceny                            70049
Other Assaults                     63570
Moving Traffic Violations          53681
Liquor Laws                        52483
Prostitution/Allied                40801
Disorderly Conduct                 37387
Weapon (carry/poss)                31228
Robbery                            26649
Vehicle Theft                      25274
Burglary                           24388
Sex (except rape/prst)             13429
Pre-Delinquency                    11897
Fraud/Embezzlement                 11443
Against Family/Child                7457
Forgery/Counterfeit                 6996
Non-Criminal Detention              6908
Receive Stolen Property             6726
Disturbing the Peace                3149
Rape                                3002
Gambling        

In [6]:
#Extract relevant numeric and categorical features in separate dataframes 


In [7]:
#filter to only DUI crimes
#create a mask --> where CGD == "Driving Under Influence", extract driving under influence data 
dui_data = arrest_data[arrest_data["Charge Group Description"] == "Driving Under Influence"]

In [8]:
#how much data after filtering
dui_data.shape 

#(121528, 25)
#array has two dimensions: 121528 entries and 25 columns

(121528, 25)

In [9]:
dui_data

,Report ID,Report Type,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Disposition Description,Address,Cross Street,LAT,LON,Location,Booking Date,Booking Time,Booking Location,Booking Location Code
948,150326444,RFC,12/12/2015,2000.0,3,Southwest,361,19,M,B,22.0,Driving Under Influence,M,23152(E)VC,DUI .04 OR MORE BAC PASS FOR HIRE,MISDEMEANOR COMPLAINT FILED,LA BREA,RODEO,34.0183,-118.3556,POINT (-118.3556 34.0183),NaN,NaN,NaN,NaN
973,151817927,RFC,09/25/2015,2020.0,18,Southeast,1801,41,F,B,22.0,Driving Under Influence,M,23152(E)VC,DUI .04 OR MORE BAC PASS FOR HIRE,MISDEMEANOR COMPLAINT FILED,MANCHESTER,HOOVER,33.9601,-118.2881,POINT (-118.2881 33.9601),NaN,NaN,NaN,NaN
1131,162106165,RFC,02/19/2016,2220.0,21,Topanga,2105,40,M,W,22.0,Driving Under Influence,M,23152(E)VC,DUI .04 OR MORE BAC PASS FOR HIRE,MISDEMEANOR COMPLAINT FILED,CANOGA AV,CHASE ST,34.2246,-118.5976,POINT (-118.5976 34.2246),NaN,NaN,NaN,NaN
4297,100601250,RFC,11/01/2010,215.0,6,Hollywood,636,22,F,O,22.0,Driving Under Influence,M,23152(A)VC,DRUNK DRIVING ALCOHOL/DRUGS,MISDEMEANOR COMPLAINT FILED,CAHUENGA BL,HOLLYWOOD BL,34.1016,-118.3297,POINT (-118.3297 34.1016),NaN,NaN,NaN,NaN
4307,100601358,RFC,12/18/2010,140.0,6,Hollywood,645,26,M,W,22.0,Driving Under Influence,M,23152(A)VC,DRUNK DRIVING ALCOHOL/DRUGS,MISDEMEANOR COMPLAINT FILED,HOLLYWOOD BL,HIGHLAND AV,34.1016,-118.3387,POINT (-118.3387 34.1016),NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1320924,5765337,BOOKING,10/05/2019,212.0,1,Central,119,22,M,H,22.0,Driving Under Influence,M,23152(A)VC,DRUNK DRIVING ALCOHOL/DRUGS,NaN,RIVERSIDE,EADS,34.0577,-118.2318,POINT (-118.2318 34.0577),10/05/2019 12:00:00 AM,351.0,77TH ST,4212.0
1320951,5810338,BOOKING,11/28/2019,1915.0,18,Southeast,1834,45,F,B,22.0,Driving Under Influence,M,23152(A)VC,DRUNK DRIVING ALCOHOL/DRUGS,MISDEMEANOR COMPLAINT FILED,108TH,TOWNE,33.9398,-118.2611,POINT (-118.2611 33.9398),11/28/2019 12:00:00 AM,2122.0,77TH ST,4212.0
1320959,5822397,BOOKING,12/14/2019,302.0,15,N Hollywood,1566,28,M,W,22.0,Driving Under Influence,M,23152(A)VC,DRUNK DRIVING ALCOHOL/DRUGS,NaN,RIVERSIDE,VINELAND,34.1524,-118.3703,POINT (-118.3703 34.1524),12/14/2019 12:00:00 AM,428.0,VALLEY - JAIL DIV,4279.0
1321067,3755468,BOOKING,10/26/2013,2155.0,6,Hollywood,647,22,M,B,22.0,Driving Under Influence,M,23152(A)VC,DRUNK DRIVING ALCOHOL/DRUGS,FELONY COMPLAINT FILED,HOLLYWOOD,ARGYLE,34.1016,-118.3252,POINT (-118.3252 34.1016),10/27/2013 12:00:00 AM,4.0,HOLLYWOOD,4206.0


In [10]:
#1) Data Cleaning 

def clean_data(orig_df): 
    
  #make a copy of original dui data 
    df = orig_df.copy()
    
  #extract rows where report ID is not NaN
    df = df[df["Report ID"].notna()]
    
 #drop duplicate rows 
    df = df.drop_duplicates()
    
 #add more to clean_data() as needed when doing EDA
    
    return df

        

In [11]:
dui_datac = clean_data(dui_data)
arrest_datac = clean_data(arrest_data)

In [12]:
#Unique charge descriptions for Driving Under Influence Arrests
dui_datac["Charge Description"].unique()

array(['DUI .04 OR MORE BAC PASS FOR HIRE', 'DRUNK DRIVING ALCOHOL/DRUGS',
       'DUI ALCOHOL CAUSING INJURY', 'DRUNK DRIVING .10 OR ABOVE',
       'DUI ALCOHOL/DRUGS ON BICYCLE', 'DUI OF ANY DRUG',
       'UNDER 21 DRIVING VEH W/BLOOD ALCOHOL .01+',
       'DUI OF COMBINED DRUG & ALCOHOL INFLUENCE',
       'DRUNK DRIVING ADDICT DRIVE VEHICLE', 'DUI ALCOHOL/DRUG W/PRIORS',
       'DUI WITH PRIOR CONVITIONS', 'DUI ALCOHOL/0.08 W/PRS',
       'DUI .08 ALCOHOL CAUSING INJURY', 'FELONY FOR FOURTH DUI',
       'DUI W/PRIOR SPEC CONVICTIONS',
       'MINOR DRIVE W/BLOOD ALCOHOL OF 0.05%+',
       'DRIVING COMMERCIAL VEH W/.04 BLOOD/ALCOHL', 'DRUNK DRIVING',
       'OPERATE MOTOR SCOOTR WHILE UNDER INFLUENC', 'DUI DRUGS',
       'DUI/BOAT OR WATERCRAFT', 'DUI OF A DRUG CAUSING INJURY',
       'DUI W/PRIOR SEPC CONVICTIONS', 'DUI W/PRIOR SPECIFIC CONVICTIONS',
       'DUI WITH PRIOR CONVICTIONS',
       'DUI OF ALCOHOL & DRUG CAUSING INJURY', 'ATTEMPT DRUNK DRIVING'],
      dtype=object)

In [13]:
#Univariate Analysis -- distribution of types of DUI arrests or overall crime data 


In [14]:
#Bi-Variate Analysis

In [15]:
#arrest_datac["Charge Description"]
#selects each charge description entry (each row)

In [16]:
#split data by area name (each dataframe corresponding to an area name)
#select charge description column and display # of unique charge description values for the area name
#divide by #arrest_datac.shape[0] which gives you total number of rows (arrests) in the area (--> actually gives total # of rows of cleaned arrest data set)
#multiply by 100 to get the proportions

#trends of DUI arrests over the week, year, or days for a certain area 

#split data by area name (each dataframe corresponding to an area name)
#select charge description column and display # of unique charge description values for the area name

arrest_area = arrest_datac.groupby("Area Name")["Charge Group Description"].value_counts() / arrest_datac.shape[0] * 100

#add this line to get all rows to show up in dataframe 
pd.set_option('display.max_rows', None)

#convert arrest by area panda series to 
df = pd.DataFrame(arrest_area)
df

#next list top 10 charge descriptions for each area



Charge Group Description
Area Name   Charge Group Description                                
77th Street Miscellaneous Other Violations                  0.793638
            Narcotic Drug Laws                              0.613868
            Aggravated Assault                              0.595928
            Driving Under Influence                         0.532574
            Prostitution/Allied                             0.425393
            Moving Traffic Violations                       0.380810
            Other Assaults                                  0.366882
            Weapon (carry/poss)                             0.299289
            Robbery                                         0.224807
            Larceny                                         0.223672
            Vehicle Theft                                   0.208079
            Burglary                                        0.156684
            Drunkeness                                      0.148055
            Sex (except rape/prst)                          0.083262
            Non-Criminal Detention                          0.051925
            Against Family/Child                            0.050865
            Fraud/Embezzlement                              0.043069
            Receive Stolen Property                         0.038906
            Forgery/Counterfeit                             0.030201
            Homicide                                        0.028158
            Pre-Delinquency                                 0.025206
            Rape                                            0.023162
            Disorderly Conduct                              0.017258
            Liquor Laws                                     0.015063
            Gambling                                        0.007645
            Disturbing the Peace                            0.004542
            Federal Offenses                                0.001665
Central     Miscellaneous Other Violations                  2.183733
            Narcotic Drug Laws                              1.846447
            Disorderly Conduct                              1.500607
            Drunkeness                                      0.839507
            Liquor Laws                                     0.545895
            Other Assaults                                  0.424939
            Larceny                                         0.399733
            Driving Under Influence                         0.392391
            Aggravated Assault                              0.369002
            Moving Traffic Violations                       0.244941
            Robbery                                         0.133749
            Sex (except rape/prst)                          0.100368
            Weapon (carry/poss)                             0.089317
            Burglary                                        0.086138
            Fraud/Embezzlement                              0.065626
            Vehicle Theft                                   0.064339
            Prostitution/Allied                             0.063203
            Forgery/Counterfeit                             0.056542
            Receive Stolen Property                         0.018318
            Rape                                            0.016350
            Disturbing the Peace                            0.015820
            Gambling                                        0.013095
            Against Family/Child                            0.012943
            Non-Criminal Detention                          0.011581
            Homicide                                        0.011354
            Pre-Delinquency                                 0.006207
            Federal Offenses                                0.001362
Devonshire  Miscellaneous Other Violations                  0.541808
            Larceny                                         0.512666
            Narcotic Drug Laws            

In [17]:
dui_data.head()

,Report ID,Report Type,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Disposition Description,Address,Cross Street,LAT,LON,Location,Booking Date,Booking Time,Booking Location,Booking Location Code
948,150326444,RFC,12/12/2015,2000.0,3,Southwest,361,19,M,B,22.0,Driving Under Influence,M,23152(E)VC,DUI .04 OR MORE BAC PASS FOR HIRE,MISDEMEANOR COMPLAINT FILED,LA BREA,RODEO,34.0183,-118.3556,POINT (-118.3556 34.0183),NaN,NaN,NaN,NaN
973,151817927,RFC,09/25/2015,2020.0,18,Southeast,1801,41,F,B,22.0,Driving Under Influence,M,23152(E)VC,DUI .04 OR MORE BAC PASS FOR HIRE,MISDEMEANOR COMPLAINT FILED,MANCHESTER,HOOVER,33.9601,-118.2881,POINT (-118.2881 33.9601),NaN,NaN,NaN,NaN
1131,162106165,RFC,02/19/2016,2220.0,21,Topanga,2105,40,M,W,22.0,Driving Under Influence,M,23152(E)VC,DUI .04 OR MORE BAC PASS FOR HIRE,MISDEMEANOR COMPLAINT FILED,CANOGA AV,CHASE ST,34.2246,-118.5976,POINT (-118.5976 34.2246),NaN,NaN,NaN,NaN
4297,100601250,RFC,11/01/2010,215.0,6,Hollywood,636,22,F,O,22.0,Driving Under Influence,M,23152(A)VC,DRUNK DRIVING ALCOHOL/DRUGS,MISDEMEANOR COMPLAINT FILED,CAHUENGA BL,HOLLYWOOD BL,34.1016,-118.3297,POINT (-118.3297 34.1016),NaN,NaN,NaN,NaN
4307,100601358,RFC,12/18/2010,140.0,6,Hollywood,645,26,M,W,22.0,Driving Under Influence,M,23152(A)VC,DRUNK DRIVING ALCOHOL/DRUGS,MISDEMEANOR COMPLAINT FILED,HOLLYWOOD BL,HIGHLAND AV,34.1016,-118.3387,POINT (-118.3387 34.1016),NaN,NaN,NaN,NaN


In [18]:
#use only dui data 

#goal: calculate the proportion of DUI arrests (out of total # arrests for each area)

#for each charge description in each area, count the number of rows 
#dui_data dataframe contains just DUI arrests for all areas 

dui_datac = dui_datac.groupby('Area Name')['Charge Description'].count().reset_index(name = 'count')
#counting each type of charge description for each area 
#.reset_index() turns that into a new dataframe and name the aggregate "count" 
#go to this column called count --> 

dui_datac

,Area Name,count
0,77th Street,7036
1,Central,5184
2,Devonshire,4376
3,Foothill,6325
4,Harbor,2989
5,Hollenbeck,3823
6,Hollywood,9613
7,Mission,6376
8,N Hollywood,12331
9,Newton,6178


In [19]:
dui_datac.loc[dui_datac['Area Name'] == "Southwest"].iloc[0, 1]
#go into dui_datac and locating all the counts of southwest under area name 
#iloc can take in index , loc can take in column names/strings
#locating row 0, column 1 and pulls out that value

5426

In [ ]:
DUI_ratio = {"Area Name": [], "DUI Ratio": []} #create a dictionary of DUI ratios and area name
#create key that corresponds to column name, and value that corresponds to the other column name

for area in dui_datac['Area Name'].unique(): #iterating through each unique area, calculate the ratio of DUI arrests to total arrests in the area 
    dict_ratio = (dui_datac.loc[dui_datac['Area Name'] == area].iloc[0, 1]) / len(arrest_datac[arrest_datac['Area Name'] == area]) #What is this line doing??? 
    #taking count of the area --> iloc[1] pulls out the data and divide by total 
    DUI_ratio["Area Name"].append(area) #go to the key area name and append the key's values --- value for that key is a list of values
    DUI_ratio["DUI Ratio"].append(dict_ratio) #
    
    #key: area name, value: the corresponding proportion of DUI arrests for that area 
    

In [ ]:
DUI_ratiodf = pd.DataFrame.from_dict(DUI_ratio) #create dataframe from a dictionary
DUI_ratiodf


In [ ]:
#To create a df of Ratios for each type of charge description (for later)
 
#the put the for loop into a function
#anywhere where it says DUI --> input the type of charge
#nest all of that for loop within another for loop that loops through all types of charges 

#create an empty dataframe at the very start and loop it through each type of charge --> loop through each area and create individual charge dataframe 
#and merge into an empty dataframe

In [ ]:
#sort DUI_ratiodf by DUI Ratio (in decreasing order for DUI ratio from all years) 

#Among all areas, N Hollywood, West Valley, Foothill, Van Nuys, Wilshire, Devonshire, West La, and Olympic had > 10% of arrests being DUI arrests over the years 
#N. Hollywood had a exceptionally high DUI ratio of almost 20% of all arrests being DUI arrests, I wonder what the trend for DUI arrests looks like over the years (e.g.
#has it been decreasing? Or staying about the same?)
#What is the DUI arrest trend like for other areas? 

DUI_ratiodf.sort_values("DUI Ratio", ascending = False)

In [ ]:
#reset dui_datac to include all the other columns
dui_datac = clean_data(dui_data)
dui_datac.head()

In [ ]:
#line graph 
#1) trends of num of DUI arrests against year for North Hollywood area 
#2) trends of DUI arrests against year for all the other areas 
#3) map plot and plot arrests for each area on a map 
    #have a slider to slide along years --> arrests pop up for one area

#convert Arrest date column to a dt attribute so dates show up as Year - month - day
dui_datac["Arrest Date"] = pd.to_datetime(dui_datac["Arrest Date"])

#extract year from Arrest Date --> add a column to dui_datac df of year of arrest 
dui_datac["Arrest Year"] = dui_datac["Arrest Date"].dt.year
 
#make a dataframe consisting of dui arrest data grouped by area name and arrest year 

#NEXT STEP: put this in graph form but not completely sure how to do that
duiyear = dui_datac.groupby(["Area Name", "Arrest Year"]).size().reset_index(name = "Count")
duiyear.head()


In [ ]:
#duiyear.plot(kind = "barh")
dict = {}

#putting area names into a dictionary
#enumerate gives you a list of tuples
for count, name in enumerate(duiyear['Area Name'].unique()):
    dict[count] = name #originally: dict[count+1] = name
    
dict

In [ ]:
from matplotlib import pyplot as plt

#graphing num of dui arrests against year for each area name to see trend of dui arrests 
#not sure how to set title for each graph, the x and y axis 

def line_graph(df):
    
    nrows = 3
    ncols = 7
    fig, ax = plt.subplots(nrows, ncols, figsize = (25, 5), sharey = True)
    count = 0
    
    #ax[0].set(yaxis = "Year") #set_ylabel, or set_xlabel 
    
    for row in np.arange(0, nrows): #iterate through each row
        for col in np.arange(0, ncols): #iterate through each col
            area_df = df[df["Area Name"] == dict[count]] #extract data for the specific area name and set to new data frame
            ax[row][col].plot(area_df["Arrest Year"], area_df["Count"])
            ax[row][col].set_title(dict[count])
            ax[row][col].set_xlabel("Year")
            ax[row][col].set_ylabel("Number of Arrests")
            count+=1
    
    #fig.text(0.5, 0.005, 'Year', ha='center', va='center', fontsize = 12)
    #plt.ylabel("Number of Arrests")
    #fig.text(0.001, 0.5, 'Number of Arrests', ha='center', va='center', fontsize = 12, rotation = "vertical") #the lower the x, closer to left; the lower the y, closer to bottom

    fig.tight_layout()
            
line_graph(duiyear)

#note anomolies, some trends that are consistently flat, similar trends,  
#any overall increasing, decreasing trend 
#at which year, dui arrests peaked 
#recommendations on what else to explore --> 

#to show which regions have worse DUI arrests --> make recs on allocation of resources 
#table of values
#count per year for each region to compare 
#2011-2015 sum of that against 2016-2019 (for each region)

#for cleaning: any outliers in latitude, longtitude
#can jump right into making the map to see outliers

#streamlit to deploy the map as standalone website, folium to plot for interactive map 


In [ ]:
county = gpd.read_file("/Users/anniechen/Downloads/tl_2018_06037_roads")
fig, ax = plt.subplots(figsize = (15, 15))
county.plot(ax = ax)

In [ ]:
dui_datac.head()

In [ ]:
#point objects (from latitude and longtitude) are already created in the dataframe
#now proceed to plot the points onto the map (for one year 2019)

#making the Location column --
geometry = [Point(xy) for xy in zip( dui_data["LAT"], dui_datac["LON"])] 

geo_df = gpd.GeoDataFrame(dui_datac, crs = "EPSG:4326", geometry = geometry)
geo_df.head()


In [ ]:
#plotting a map of where most dui arrests are occurring starting for a specific year (2019)
fig, ax = plt.subplots(figsize = (15, 15))
county.plot(ax = ax, alpha = 0.4, color = "grey")
geo_df = geo_df[geo_df["Arrest Year"] == "2019"]

geo_df.plot(ax = ax, markersize = 20, color = "blue", column = "Location")

#plt.legend(prop = {"size": 15})


In [ ]:
#get latitudes and longtitudes as columns
fig, ax = plt.subplots(figsize = (15, 15))
county.plot(ax = ax, alpha = 0.4, color = "grey")

dui_datac = dui_datac[(dui_datac["LON"] < -100) & (dui_datac["LAT"] > 30)]

#converts raw LAT and LON into geopandas coordinate system (same as list comprehension above)
data_points = gpd.GeoDataFrame(dui_datac, geometry=gpd.points_from_xy(dui_datac.LON, dui_datac.LAT))

#plot where you intend to plot 
data_points.plot(ax = ax)

#filter out longtitudes < -100, latitude > 30 --> keep 


#plt.hist(geo_df.loc[:, "LAT"].values)

In [ ]:

#Scratch work
#dui_data['Area Name'].unique #extract the values in area name column and output the unique area names

In [ ]:
#Scratch work
#for area in dui_data['Area Name'].unique:
   # dui_data['Area Name'] == area

In [ ]:
#Scratch work 
#visualizations -- show distribution of variables of interest 
#variables of interest 
#plotting one of variables in arrest data against time

#how would I go about doin gthat
#plotting number of dui arrests for each region against time
#plotting number of dui arrests collectively against time --> see if there are any anomolies there 
